# Extração dos dados das estações e S4
___

Os dados de estações e S4 estão organizados em um servidor postgresql rodando na minha maquina local. Para permitir a utilização por outros, este notebook, além de extrair os dados do postgresql vai gerar um csv, contendo as informações sobre as estações brutas. Assim, como um dataframes pandas, um por estação, comprimido usando o formatado xz, o que forneceu a extensão "pkl.xz".

As estações válidas são aquelas que apresentam algum dado de S4, pois existem estações que não coletaram dado nenhum durante o período de aquisição.

Note assim, que este notebook não executará em um máquina onde os dados não estejam armazenados no postgresql.

In [1]:
import aacgmv2
import os
import sys
import psycopg2

import datetime as dt
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from mpl_toolkits.basemap import Basemap
from itertools import chain
from tqdm import tqdm

from utils import local_data

%matplotlib inline

In [2]:
force = True

In [3]:
conn = psycopg2.connect(database="data-research",
                        user="data-research",
                        password="data-research",
                        host="localhost",
                        port=5432)

In [4]:
path = "./data/stations.csv"

if os.path.exists(path) and not force:
    df_station = pd.read_csv(path)
else:    
    df_station = pd.read_sql_query('select * from station;', con=conn)
    df_station.to_csv(path)

#df_station = df_station.drop(columns=["startoperation", "stopoperation", "alias", "network", "Unnamed: 0"])    

In [5]:
query = '''select scintillation.eventdate, scintillation.s4 
           from scintillation inner join station on 
           scintillation.station = station.identificationstation 
           where station.identificationstation = '%s' and elevation > 30.0 
           order by scintillation.eventdate; 
        '''
drop_aux = []
select_station_and_df_s4 = []
for i in tqdm(df_station.itertuples()):
    station = i.identificationstation
    path = "./data_s4/df_" + station + ".pkl.xz" 
    if not os.path.exists(path) or force:
        df_aux = pd.read_sql_query(query % station, con=conn)
        if len(df_aux) == 0:
            drop_aux.append(station)
        else:
            df_aux.to_pickle(path, compression='xz')

54it [33:06, 36.80s/it]


In [6]:
df_station = df_station[~df_station['identificationstation'].isin(drop_aux)]
df_station.to_pickle(os.path.join(local_data, 'df_station.pkl'))

In [7]:
df_station

identificationstation alias    height   latitude  longitude     network  \
7                    cpa  null   580.000 -22.410000 -45.000000        lisn   
8                     28  man3       NaN  -3.100000 -59.976000  septentrio   
9                     20  cg01       NaN -20.497000 -54.615000  septentrio   
10                    32  bacg       NaN -20.497000 -54.615000  septentrio   
11                   nta  null     0.000  -5.836162 -35.121000        lisn   
12                   sj2  null   593.440 -23.207600 -45.859800        lisn   
14                    16  galh       NaN -22.823000 -43.238000  septentrio   
15                    22  gw01       NaN -22.823000 -43.238000  septentrio   
16                   cub  null   278.000 -15.555200 -56.069800        lisn   
17                   dou  null   756.120 -22.110000 -54.550000        lisn   
18                    24  fort       NaN  -3.742000 -38.539000  septentrio   
19                    34  gltw       NaN -22.823000 -43.238000  septentrio   
20                    33  eear       NaN -22.789000 -45.220000  septentrio   
21                   ios  null     0.000 -14.470000 -39.100000        lisn   
22                   imp  null   134.720  -5.310000 -47.280000        lisn   
23                    25  inco       NaN -22.318000 -46.329000  septentrio   
24                     1  maca       NaN -22.823000 -41.785700  septentrio   
25                    11  mac2       NaN -22.823000 -41.785700  septentrio   
26                     3  palm       NaN -10.200000 -48.312000  septentrio   
27                   pln  null   434.340  -9.230000 -40.300000        lisn   
28                    30  afae       NaN -21.989000 -47.334000  septentrio   
29                     4  poal       NaN -30.071000 -51.119000  septentrio   
30                     5  pru1       NaN -22.120000 -51.407000  septentrio   
31                     6  pru2       NaN -22.120000 -51.407000  septentrio   
32                    17  moru       NaN -22.120000 -51.407000  septentrio   
33                    18  prut       NaN -22.120000 -51.407000  septentrio   
34                    19  pru3       NaN -22.120000 -51.407000  septentrio   
36                     8  sjcu       NaN -23.207000 -45.859000  septentrio   
37                    14  sjce       NaN -23.207000 -45.859000  septentrio   
39                    29  slma       NaN  -2.574000 -44.209000  septentrio   
40                   afl  null   267.000  -9.900000 -55.900000        lisn   
41                    31  afaw       NaN  -9.870000 -56.104000  septentrio   
42                   bhz  null   858.000 -19.868500 -43.954200        lisn   
43                   bov  null    69.180   2.833920 -60.695380        lisn   
47                    26  ufba       NaN -13.001000 -38.508000  septentrio   
48                   tfe  null     0.057  -3.180000 -64.440000        lisn   
49                   bsa  null  1050.000 -15.764200 -47.869400        lisn   
50                   pvh  null  1130.000  -8.490000 -63.550000        lisn   
51                   sta  null   110.100 -29.712591 -53.717206        lisn   
52                    15  man2       NaN  -3.100000 -59.976000  septentrio   

   state                                      city startoperation  \
7     sp                        cachoeira paulista           None   
8     am                                    manaus           None   
9     ms                              campo grande           None   
10    ms                             campos grande           None   
11    rn                                     natal           None   
12    sp                       sao jose dos campos           None   
14    rj         rio de janeiro (aeroporto galeão)           None   
15    rj         rio de janeiro (aeroporto galeão)           None   
16    mt                                    cuiaba           None   
17    ms                                  dourados           None   
18    ce                                 fort